# Coding Explanation Test Material

<br>



---



In [ ]:
#@title
# Import libraries
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *

import pandas as pd
import numpy as np

Below I have included the full code for two simulations: the bikeshare and queue theory models.   On the test, I will choose one of these, and ask 12-15 questions about specific lines in the code.   The exact phrasing of these questions might vary, but they will all essentially ask, "What does this line do?"  

<br> A good answer will explain not only what the code does in a literal sense ("It creates an empty Series called 'results'") but also how it functions in the simulation ("'Results' is later used to store the values of the state variables at each time step").


## The bikeshare model

In [ ]:
# Function definitions
def run_simulation(p1, p2, iAug, iMol, num_steps):
    state = pd.Series(dict(augie=iAug,moline=iMol,augie_empty=0,moline_empty=0,
                           clock=0),name="Number of Bikes")
    for i in range(num_steps):
        change_func(state, p1, p2)

    return state


def run_multiple_simulations(p1, p2, iAug, iMol, num_steps,num_runs):
    unhappy = pd.Series([],dtype=object)
    unhappy.name = 'Number of Unhappy Customers'
    unhappy.index.name = 'Simulation Number'

    for i in range(num_runs):
        final_state = run_simulation(p1, p2, iAug, iMol, num_steps)
        unhappy[i] = final_state.augie_empty + final_state.moline_empty
    return unhappy

def sweep_bikeshare(p1, p2, total_bikes, num_steps,num_runs):
    unhappy_ave = pd.Series([],dtype=object)
    unhappy_ave.name = 'Average Number of Unhappy Customers'
    unhappy_ave.index.name = 'Initial Bikes at Augie'

    for i in range(total_bikes+1):
        unhappy = run_multiple_simulations(p1,p2,i,total_bikes-i,num_steps,num_runs)
        unhappy_ave[i]= unhappy.mean()
        unhappy_ave.plot(xlabel=unhappy_ave.index.name, ylabel=unhappy_ave.name,
            title='Augie-Moline Bikeshare', color='C1',legend=False,
            linestyle='--',marker='.');
    return unhappy_ave

def change_func(state, p1, p2):
    if flip(p1):
        if state.augie == 0:
            state.augie_empty += 1
        else:
            state.augie -= 1
            state.moline += 1

    if flip(p2):
        if state.moline == 0:
            state.moline_empty += 1
        else:
            state.moline -= 1
            state.augie += 1


# Runtime code
p1 = 0.5; p2 = 0.4; total_bikes = 12
num_steps = 60; num_runs = 100

unhappy_ave = sweep_bikeshare(p1, p2, total_bikes, num_steps,num_runs)

## Queue Theory Model

In [ ]:
def mult_runs(lam,mu,num_sims,num_steps,change_func):                           #A
    mean = pd.Series([],dtype = object)                                         #B
    for i in range(num_sims):
        line = pd.Series(dict(cust=0,cust2=0),name='Customers in line')
        global_results = run_simulation(line, lam,mu,num_steps, change_func)
        metrics = compute_metrics(global_results,lam)
        mean[i] = metrics[0]
    return mean

def run_simulation(state,lam,mu,num_steps,change_func):
    results = pd.Series([],dtype=object)
    results[0] = state.sum()

    for t in range(num_steps):
        state = change_func(state,lam,mu)
        results[t+1] = state.sum()

    results.plot(label='customers in line',color='C3', title='Customers in line',
                 xlabel='time', ylabel='Number of customers');
    return results

def compute_metrics(results, lam):
    L = results.mean()
    M = results.max()
    W = L / lam
    return L, M, W

# Change function for two queues and two servers
def change_func_2q_2s(state,lam,mu):

    if state.cust > 0:
        if flip(mu):
            state.cust -= 1
    if state.cust2 > 0:
        if flip(mu):
            state.cust2 -= 1

    if flip(lam):
        if state.cust > state.cust2:
            state.cust2 += 1
        else:
            state.cust += 1
    return state

lam = 0.5; mu = 0.3; num_sims = 25; num_steps = 120
mean = mult_runs(lam, mu, num_sims, num_steps, change_func_2q_2s)
mean.mean()